In [1]:
import torch
import pathlib
import matplotlib.pyplot as plt
from src.make_dataset import DataModule as DM
from src.train_model import deeplab, segformer
from src.model2 import segformer2
from pytorch_lightning import Trainer
from src.make_dataset import DeloitteDataSet

/Users/erikriise/anaconda3/envs/dl_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download weights from google drive (about 6.8GB)

In [11]:
import gdown

url = "https://drive.google.com/uc?id=1gTbyaXJJHAeHO7BtUSl3zHmX9Ff0dcFG"

output_file = "weights.zip"

gdown.download(url, output_file, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1gTbyaXJJHAeHO7BtUSl3zHmX9Ff0dcFG
From (redirected): https://drive.google.com/uc?id=1gTbyaXJJHAeHO7BtUSl3zHmX9Ff0dcFG&confirm=t&uuid=ccd620dc-2e20-4d72-a1fc-865022b3c8bf
To: /Users/erikriise/Documents/python_projects/02456-dl/weights.zip
100%|██████████| 6.81G/6.81G [02:05<00:00, 54.4MB/s]


'weights.zip'

In [12]:
import zipfile

zip_file_path = "weights.zip"

destination_folder = "weights"

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(destination_folder)

In [2]:
weights_folder = pathlib.Path("weights/weights")
weights_paths = list(weights_folder.glob("*"))
weights_paths = [str(path) for path in weights_paths]

## Reproducing the results from experiments.

In [3]:
def test(wts):
    if ".DS_Store" in wts:
        return
    # if mps is available, use it
    if torch.cuda.is_available():
        device = "gpu"
    else:
        device = "cpu"

    if "syn" in wts:
        print("Using synthetic data")
    else:
        print("Using real data only")

    if "focal" in wts:
        print("Using focal loss")
    elif "Lovasz" in wts:
        print("Using lovasz loss")
    else:
        print("Using cross entropy loss")

    if "50" in wts:
        print("Using Deeplabv3 resnet50 backbone")
        model = deeplab(learning_rate=3e-4, num_classes=9, backbone="resnet50")
    elif "101" in wts:
        print("Using Deeplabv3 resnet101 backbone")
        model = deeplab(learning_rate=3e-4, num_classes=9, backbone="resnet101")
    else:
        print("Using Segformer")
        model = segformer(learning_rate=3e-4, num_classes=9)

    if "more-parts" in wts:
        print("Using more parts")
        model = segformer2(learning_rate=3e-4, num_classes=28)

    trainer = Trainer(
        accelerator=device,
    )
    batch_size = 32
    dm = DM(DeloitteDataSet, batch_size=batch_size, augment=True)
    dm.setup()

    trainer.test(model, dm, ckpt_path=str(list(pathlib.Path(wts).glob("*"))[0]))

In [5]:
for wts in weights_paths:
    test(wts)

Using synthetic data
Using cross entropy loss
Using Segformer


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at weights/weights/segformer-with-syn-and-augs/epoch=2-step=1338.ckpt
Lightning automatically upgraded your loaded checkpoint from v1.9.0 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint weights/weights/segformer-with-syn-and-augs/epoch=2-step=1338.ckpt`
Loaded model weights from the checkpoint at weights/weights/segformer-with-syn-and-augs/epoch=2-step=1338.ckpt


Testing DataLoader 0: 100%|██████████| 30/30 [01:59<00:00,  0.25it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          Dice              0.8334762141812412
       dice Bumper          0.8184349226916948
       dice Frame           0.7852013273240552
     dice Front door        0.9032969104606394
        dice Hood           0.9137620684012436
     dice Rear door         0.8570938456666315
 dice Rear quarter panel    0.6907173373965523
     dice Trunk lid         0.8658270873278717
          mIoU              0.7206824202597668
       mIoU Bumper          0.6926702036219212
       mIoU Frame           0.6463633398564905
     mIoU Front door        0.8236476390707023
        mIoU Hood           0.8412172341066586
     mIoU Rear door         0.749

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at weights/weights/dl50-with-augs/epoch=85-step=602.ckpt
Lightning automatically upgraded your loaded checkpoint from v1.9.0 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint weights/weights/dl50-with-augs/epoch=85-step=602.ckpt`
Loaded model weights from the checkpoint at weights/weights/dl50-with-augs/epoch=85-step=602.ckpt


Testing DataLoader 0: 100%|██████████| 30/30 [00:05<00:00,  5.06it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          Dice              0.7913497137038377
       dice Bumper          0.7595360473049807
       dice Frame           0.7454181864481676
     dice Front door         0.887427995391705
        dice Hood           0.9076160207664276
     dice Rear door          0.803966254847867
 dice Rear quarter panel    0.6240254024144869
     dice Trunk lid         0.8114580887532284
          mIoU              0.6633420149600894
       mIoU Bumper           0.612299975065636
       mIoU Frame           0.5941566969935764
     mIoU Front door        0.7976364601265905
        mIoU Hood           0.8308580481043123
     mIoU Rear door         0.672

...

Testing DataLoader 0: 100%|██████████| 30/30 [01:56<00:00,  0.26it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          Dice              0.8031979572886441
       dice Bumper          0.8068027210884354
       dice Frame           0.7492144136736072
     dice Front door        0.8846097674962028
        dice Hood           0.9058025746798251
     dice Rear door         0.8126593987446317
 dice Rear quarter panel    0.6174449749621521
     dice Trunk lid         0.8458518503756551
          mIoU              0.6799993457378808
       mIoU Bumper          0.6761687571265679
       mIoU Frame           0.5989950810626783
     mIoU Front door        0.7930944181844369
        mIoU Hood           0.8278237123568232
     mIoU Rear door         0.684

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at weights/weights/dl50-with-syn-and-augs/epoch=11-step=2676.ckpt
Lightning automatically upgraded your loaded checkpoint from v1.9.0 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint weights/weights/dl50-with-syn-and-augs/epoch=11-step=2676.ckpt`
Loaded model weights from the checkpoint at weights/weights/dl50-with-syn-and-augs/epoch=11-step=2676.ckpt


Testing DataLoader 0: 100%|██████████| 30/30 [00:06<00:00,  4.99it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          Dice              0.8249009888825408
       dice Bumper          0.7459789392647989
       dice Frame           0.7663060369707105
     dice Front door        0.9086864529592488
        dice Hood           0.9138923427837387
     dice Rear door         0.8534124473952479
 dice Rear quarter panel    0.7061759576643214
     dice Trunk lid         0.8798547451397193
          mIoU              0.7093862093380696
       mIoU Bumper          0.5948695461521596
       mIoU Frame           0.6211475940832804
     mIoU Front door         0.832653874244738
        mIoU Hood           0.8414380809413334
     mIoU Rear door         0.744

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at weights/weights/dl101-with-syn-and-augs/epoch=11-step=5352.ckpt
Lightning automatically upgraded your loaded checkpoint from v1.9.0 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint weights/weights/dl101-with-syn-and-augs/epoch=11-step=5352.ckpt`
Loaded model weights from the checkpoint at weights/weights/dl101-with-syn-and-augs/epoch=11-step=5352.ckpt


Testing DataLoader 0: 100%|██████████| 30/30 [00:07<00:00,  3.80it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          Dice              0.8309991654095731
       dice Bumper          0.7698048911483364
       dice Frame            0.784688995215311
     dice Front door        0.9136889436849549
        dice Hood           0.9232410076042532
     dice Rear door         0.8576726574859307
 dice Rear quarter panel    0.6772106398274622
     dice Trunk lid         0.8906870229007634
          mIoU               0.719376096362512
       mIoU Bumper          0.6257583741061348
       mIoU Frame           0.6456692913385826
     mIoU Front door        0.8410932930981534
        mIoU Hood            0.85742586235577
     mIoU Rear door         0.7508